In [60]:
import pandas as pd
import os
import re

In [2]:
only_files = [f for f in os.listdir("/Users/haqn/Desktop/VLSP23-Comparative-Opinion-Mining/data/combinedData") if
              os.path.isfile(os.path.join("/Users/haqn/Desktop/VLSP23-Comparative-Opinion-Mining/data/combinedData", f))]

In [3]:
only_files

['merged_file_relabled_train.txt']

In [53]:
error = []
smentederr = []
type_of_compare = ["DIF", "EQL", "SUP+", "SUP-", "SUP", "COM+", "COM-", "COM"]
data_frame = {"content": [], "comparative": [], "subject": [], "object": [], "aspect": [], "predicate": [], "label": [], "NER": []}
pred_label = dict()
dct_index_out_of_range = {}
special_char = {'oá':'óa','oã':'õa','oả':'ỏa','oà':'òa','oạ':'ọa','oé':'óe','oè':'òe','oẻ':'ỏe','oẽ':'õe','oẹ':'ọe'}

In [61]:
def preprocessing_with_BIO(filename):
    data_row = []
    labeled_row = dict()
    data = []

    # f.close()
    with (open('/Users/haqn/Desktop/VLSP23-Comparative-Opinion-Mining/data/combinedData/' + filename, 'r', encoding="utf-8") as f):
        lines = f.readlines()
        for row in lines: 
            if row != "\n":
                split_row = row.split("\t")
                if len(split_row) == 2 or (len(split_row) == 3 and split_row[-1] == '[quintuple]\n'):
                    r = split_row[1]
                    data_frame["content"].append(r.replace("\n", ""))
                elif len(split_row) == 3 and split_row[-1] == '[spellquintuple]\n':
                    pass
                else: 
                    r = row
                r = r.replace("\n", "")
                data_row.append(r)
                row = f.readline()
    del f
    j = 0
    for i in range(len(data_row) - 1, -1, -1):
        if data_row[i] == "":
            data_row.remove(data_row[i])
    for i in range(len(data_row) - 1):
        if re.match("^\{\"subject\": \[", data_row[i + 1]) and re.match("^\{\"subject\": \[", data_row[i]):
            row = eval(data_row[i + 1])
            labeled_row[data_row[j]].append(row)
        elif re.match("^\{\"subject\": \[", data_row[i + 1]):
            j = i
            row = eval(data_row[i + 1])
            labeled_row[data_row[j]] = [row]
    for i in data_row:
        if re.match("^\{\"subject\": \[", i) is None:
            data.append(i)
    del data_row
    del j
    del row
    for i in data:
        sentence_label = []
        key = 0
        subj = []
        obj = []
        asp = []
        pred = []
        lab = []
        w = i.split(" ")
        label = [0 for k in range(len(w))]
        if i in labeled_row.keys():
            key = 1
            for k in range(len(labeled_row.get(i))):
                list_dct = labeled_row.get(i)
                if type(list_dct[k]["subject"]) == str:
                    continue
                for j in list_dct[k].keys():
                    arr = list_dct[k].get(j)
                    consecutive = []
                    if j == "label":
                        continue
                    for e in range(len(arr)):
                        consecutive.append(int(arr[e][:arr[e].find("&")]))
                        arr[e] = re.sub("^[0-9]+&&", "", arr[e])
                    s = "_".join(arr)
                    list_dct[k][j] = s
                    for q in range(len(consecutive)):
                        if j == "subject":
                            if q == 0:
                                label[consecutive[q] - 1] = 1
                            else:
                                label[consecutive[q] - 1] = 2
                        elif j == "object":
                            if q == 0:
                                label[consecutive[q] - 1] = 3
                            else:
                                label[consecutive[q] - 1] = 4
                        elif j == "aspect":
                            if q == 0:
                                label[consecutive[q] - 1] = 5
                            else:
                                label[consecutive[q] - 1] = 6
                        elif j == "predicate":
                            if q == 0:
                                label[consecutive[q] - 1] = 7
                            else:
                                label[consecutive[q] - 1] = 8
                sentence_label.append(label)
                subj.append(list_dct[k]["subject"])
                obj.append(list_dct[k]["object"])
                pred.append(list_dct[k]["predicate"])
                asp.append(list_dct[k]["aspect"])
                lab.append(type_of_compare.index(list_dct[k]["label"]) + 1)
                
            if len(sentence_label) > 1:
                full_label = []
                l = 0
                for n in range(len(sentence_label[0])):
                    for j in range(len(sentence_label) - 1):
                        if sentence_label[j][n] == sentence_label[j + 1][n]:
                            l = sentence_label[j][n]
                        elif sentence_label[j][n] != sentence_label[j + 1][n]:
                            if sentence_label[j][n] == 0:
                                l = sentence_label[j + 1][n]
                            elif sentence_label[j + 1][n] == 0:
                                l = sentence_label[j][n]
                    full_label.append(l)
                sentence_label = full_label
                del full_label
            else:
                sentence_label = label
        else:
            sentence_label = label
        data_frame["comparative"].append(key)
        data_frame["subject"].append(subj)
        data_frame["object"].append(obj)
        data_frame["predicate"].append(pred)
        data_frame["aspect"].append(asp)
        data_frame["label"].append(lab)
        data_frame["NER"].append(sentence_label)
        for j in range(len(pred)):
            if pred[j] not in pred_label.keys():
                pred_label[pred[j]] = [lab[j]]
            elif pred[j] in pred_label.keys() and lab[j] not in pred_label[pred[j]]:
                pred_label[pred[j]].append(lab[j])
    return data_frame

preprocessing_with_BIO('merged_file_relabled_train.txt')

IndexError: list assignment index out of range

In [62]:
data_frame["content"]

['Title : So sánh ảnh chụp thực tế của iPhone 13 với các thế hệ iPhone tiền nhiệm',
 'Chất lượng camera của iPhone 13 được so sánh với các thế hệ iPhone tiền nhiệm của chúng dựa trên những bức ảnh chụp đến từ người sở hữu sớm , một lần nữa khẳng định việc nâng cấp đời smartphone mới là vô cùng xứng đáng !',
 'So sánh ảnh chụp của iPhone 13',
 'Trang MacRumors gần đây đã post những bức ảnh được chụp bởi iPhone 13 với iPhone 11 Pro , iPhone 8 và iPhone 6 và đi đến kết luận rằng nếu bạn yêu thích chụp ảnh và đang sử dụng đời máy cũ thì iPhone 13 đáng để nâng cấp .',
 'Cùng ngắm nhìn những bức ảnh thực tế được chụp bởi những người sở hữu iPhone 13 sớm nhất tại các nước ở dưới đây .',
 'alt :',
 'des : So sánh camera các đời iPhone chụp ảnh đường hoàng hôn .',
 'Sẽ không có gì ngạc nhiên khi iPhone 13 vượt trội hơn iPhone 6 bởi một số khác biệt rõ rệt nhất .',
 'iPhone 8 có chất lượng ảnh tốt hơn và có cấu hình màu sắc thực sự khá giống với iPhone 13 , nhưng vẫn có sự chênh lệch rõ ràng tro

In [33]:
print(len(error))
for i in range(1):
    print(error[i])

0


IndexError: list index out of range

In [25]:
import csv  
# name of csv file  
filename = "predicate_label_dictionary.csv"
row = []
# writing to csv file  
with open(filename, 'w', encoding="utf-8") as csvfile:  
    csvwriter = csv.writer(csvfile)  
    csvwriter.writerow(["Word","Label"])  
    for i in pred_label.keys():
        row = [i]
        row.append(pred_label[i])
        csvwriter.writerows([row]) 


In [4]:
df = pd.read_csv('Data/total.csv')
compared = df[df['comparative'] == 1]

In [ ]:
smted = segmenter.word_segment(i)[0].split()
        j = 0
        space = 0
        er = 0
        temp_ner = []
        while j < len(smted):
            un = smted[j].split("_")
            for u in range(len(un)):
                for s in special_char.keys():
                    if re.match('^\w+'+s+'$',un[u]):
                        un[u] = un[u].replace(s,special_char[s])
            comparing = '_'.join(un)
            if w[j+space-er] in comparing and len(w[j+space-er]) <= len(comparing):
                temp_ner.append(sentence_label[j+space-er])
                if comparing.split('_')[0] != "":
                    unseg = comparing.split("_")
                    counter = 0
                    for k in range(len(unseg)-1):
                        if abs((sentence_label[j+space-er+k+1] - sentence_label[j+space-er+k])) > 1:
                            smentederr.append(filename+','+w.__str__()+','+smted.__str__() + ',' + str(sentence_label[j+space-er+k+1]) + ',' + str(sentence_label[j+space-er+k])+','+str(j)+','+str(k))
                            counter += 1
                    space += len(unseg)-1
            elif comparing in w[j+space-er] and len(comparing) < len(w[j+space-er]):
                aim = len(w[j+space-er])
                origin = comparing
                k = 1
                temp_ner.append(sentence_label[j+space-er])
                while len(origin) < aim:
                    origin += smted[j+k]
                    k += 1
                    temp_ner.append(sentence_label[j+space-er])
                j += k-1
                er += k-1
            j+=1
        sentence_label = temp_ner

In [11]:
d = {'content':[], 'NER':[], 'label':[]}
generror = []
for k in range(1,9):
    df_comp = df_splited[df_splited['label'] == k]
    others = df_comp[['subject','object','aspect','NER']]
    content = df_comp.to_dict(orient='list')['content']
    pred = df_comp.to_dict(orient='list')['predicate']
    i = 0
    count = 0
    while count < 10000:
        stg = content[i] 
        su = others.iloc[i]['subject']
        ob = others.iloc[i]['object']
        asp = others.iloc[i]['aspect']
        pr = pred[i]
        n = others.iloc[i]['NER']
        # p = random.choice(list(vocab['predicate'].keys()))
        # f = pr.split()
        # substitute = p.split()
        # checkpoint = finder(stg,f)
        # n = n[:checkpoint[0]]+[4 for p in range(len(substitute))]+ n[checkpoint[1]+1:]
        # stg = stg.replace(pr, p)
        if su != '' and su in stg:
            f = su.split()
            sub = random.choice(list(vocab['subject']))  
            substitute = sub.split()
            checkpoint = finder(stg,f)
            stg = stg.replace(su, sub)
            n = n[:checkpoint[0]]+[1 for p in range(len(substitute))]+ n[checkpoint[1]+1:]
            # if len(stg.split()) != len(n):
            #     a = 's'
        if ob != '' and ob in stg:
            f = ob.split()
            obj = random.choice(list(vocab['object']))
            substitute = obj.split()
            checkpoint = finder(stg, f)
            stg = stg.replace(ob, obj)
            n = n[:checkpoint[0]]+[2 for p in range(len(substitute))]+ n[checkpoint[1]+1:]
            # if len(stg.split()) != len(n):
            #     a = 's'
        if asp != '' and asp in stg:
            f = asp.split()
            aspe = random.choice(list(vocab['aspect']))
            substitute = aspe.split()
            checkpoint = finder(stg,f)
            stg = stg.replace(asp, aspe)
            n = n[:checkpoint[0]]+[3 for p in range(len(substitute))]+ n[checkpoint[1]+1:]
            # if len(stg.split()) != len(n):
            #     a = 's'
        if len(stg.split()) != len(n):
            generror.append(stg+','+n.__str__()+','+str(k))
        if stg not in content:
            d['content'].append(stg)
            d['NER'].append(n)
            d['label'].append(k)
            count += 1
        i += 1
        if i == stat[k]:
            i = 0

In [12]:
counter = [0,0,0,0,0,0,0,0,0]
count = 0
for i in range(len(generror)):
    counter[int(generror[i][-1])] += 1
print(counter)
print(stat)

[0, 2081, 1714, 1866, 3948, 2050, 1991, 1773, 1201]
[0, 58, 286, 106, 5, 4, 499, 107, 20, 0]


In [22]:
df_generate = pd.DataFrame(data=d)
df_generate.head()

,content,NER,label
0,Riêng về dung lượng thì thực sự hiếm có cả ha...,"[0, 0, 3, 3, 0, 0, 0, 4, 4, 2, 2, 2, 2, 4, 4, ...",1
1,"chụp hình selfie cũng có sự thay đổi đáng kể ,...","[3, 3, 3, 0, 0, 4, 4, 4, 4, 4, 0, 0, 0, 1, 1, ...",1
2,"Rõ ràng , sẽ có một khoảng cách về dung lượng ...","[0, 0, 0, 0, 4, 4, 4, 4, 0, 3, 3, 3, 3, 0, 3, ...",1
3,"Rõ ràng , sẽ có một khoảng cách về tốc độ xử l...","[0, 0, 0, 0, 4, 4, 4, 4, 0, 3, 3, 3, 3, 0, 3, ...",1
4,Nếu như trên biến thể ultra và các thương ...,"[0, 0, 0, 1, 1, 1, 0, 2, 2, 2, 2, 0, 0, 4, 4, ...",1


In [27]:
df_shuffled = df_generate.sample(len(df_generate), random_state=1)
df_shuffled.to_csv('Data/generated_data.csv')
df_shuffled.head()

,content,NER,label
59591,kính gorilla ở cả hai mặt trên hai bộ vi xử lý...,"[3, 3, 3, 3, 3, 3, 0, 1, 1, 1, 1, 1, 1, 0, 0, ...",8
62869,các thông số kỹ thuật sạc trên samsung z fold ...,"[3, 3, 3, 3, 3, 3, 0, 1, 1, 1, 1, 0, 0, 4, 4, ...",8
60553,camera selfie galaxy s22 ultra 5g đúng như tên...,"[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",8
10401,des : Nhìn chung camera tele không có sự khác ...,"[0, 0, 0, 0, 1, 1, 4, 4, 4, 4, 4, 4, 4]",2
22102,"Không những vậy , chiếc điện thoại gập cũng...","[0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...",3


In [26]:
counter = [0,0,0,0,0,0,0,0,0]
for i in range(len(d['content'])):
    counter[d['label'][i]] += 1
print(counter)

[0, 7919, 8286, 8134, 6052, 7950, 8009, 8227, 8799]


In [20]:
gen_data = zip(d['content'],d['NER'],d['label'])
random.shuffle(gen_data)
c = 0
wrt = {'content':[],'NER':[],'label':[]}
for c, n, l in gen_data:
    if len(c.split()) != len(n):
        c+=1
    else:
        wrt['content'].append(c)
        wrt['NER'].append(n)
        wrt['label'].append(l)

print(c)

TypeError: object of type 'zip' has no len()

In [17]:
print(gen_data)

In [15]:
for i in range(5):
    print(d['content'][i],d['NER'][i],len(d['content'][i].split()), len(d['NER'][i]))

Tuy nhiên chúng vẫn có những điểm đều được trang bị như camera selfie của hai model này be shown so với màn hình 6,8 inch của biến thể ultra và hỗ trợ sạc cũng có sự đều được trang bị . [0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 0, 3, 3, 3, 0, 0, 1, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 3, 3, 3, 3, 0, 0, 0, 4, 4, 0] 42 43
phần cứng camera sau cũng có không có nhiều thay đổi , đối với iphone 13 pro max lên đến 6GB . [3, 3, 3, 3, 0, 0, 4, 4, 4, 4, 4, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0] 22 22
Rõ ràng , sẽ thú vị nhất về những bức ảnh , khả năng đồ họa và khả năng tiết kiệm năng lượng giữa một chiếc máy cao cấp . [0, 0, 0, 0, 4, 4, 4, 0, 3, 3, 3, 0, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 1, 1, 1, 1, 1, 2, 2, 2, 0, 0, 0, 0, 4, 4, 4, 4, 0, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 1, 1, 1, 1, 1, 0] 30 64
Rõ ràng , sẽ vượt trội hơn về tốc độ xử lý , chip xử lý snapdragon 8 gen 2 của qualcomm và khả năng tiết kiệm năng lượng giữa máy . [0, 0, 0, 0, 4, 4, 4, 0, 3, 3, 3, 3, 0, 3, 

In [19]:
os.listdir()

['.idea',
 'Data',
 'Data preprocessing.ipynb',
 'label_1.txt',
 'modelUtils.py',
 'predicate_label_dictionary.csv',
 'Preprocessing.py',
 'vncorenlp',
 '__pycache__']

In [102]:
for i in range(1,len(lst)):
    name = 'ccmparison_label_'+str(i)+'.txt'
    with open(name, 'w', encoding="utf-8") as resamplingfile:
        for j in range(len(lst[i])):
            resamplingfile.write(lst[i][j]+'\n')